In [2]:
import random

## Carolus Magnus (for 2 Player)s

### Classes 

Player: 
- has a name (for convenience), castles, cubes in five colors to place, and stacks for each color
- can draw a chip (defines the order of the moves), can place cubes, can redraw cubes

Region:
- has stacks for each color, a number of castles and an owner
- can add cubes to its stacks, can check which player has the majority and can can change the castles and the owner, can merge itself with another region (local helper function)

Karl:
- has an index and can be moved

Loyalty:
- has stacks for each color
- can check which player has the majorities for which colour

One global merge function to merge two regions.

In [7]:
class Player:
    def __init__(self, name):
        self.name = name
        self.castles = 10
        
        # Cubes by color
        self.cubes = {
            'red': 0,
            'blue': 0,
            'green': 0,
            'pink': 0,
            'yellow': 0
        }

        # Give 7 random cubes at start
        available_colors = list(self.cubes.keys())
        for _ in range(7):
            color = random.choice(available_colors)
            self.cubes[color] += 1
        
        # Stacks for each color
        self.color_stacks = {
            'red': 0,
            'blue': 0,
            'green': 0,
            'pink': 0,
            'yellow': 0
        }

        # Chips 1 to 5 in a bag (shuffled)
        self.bag = [1, 2, 3, 4, 5]

        # Stack for drawn chips
        self.chip_stack = []

    def draw_chip(self, forbidden=None):
        """
        Draw a chip from the bag and move it to the chip stack.
        If the bag is empty, reshuffle the chip stack into the bag first.
        The forbidden value ensures that the chip drawn is not the forbidden one.
        """
        if len(self.bag) == 0:  # If the bag is empty, refill it from the stack
            self.bag = self.chip_stack.copy()
            self.chip_stack.clear()
            random.shuffle(self.bag)  # Shuffle to ensure randomness

        # Ensure a valid chip is drawn (not the forbidden one)
        if forbidden is not None and len(self.bag) > 1:
            available_chips = [chip for chip in self.bag if chip != forbidden]
        else:
            available_chips = self.bag

        # Draw a random chip from the available chips
        drawn_chip = random.choice(available_chips)
        
        # Move the drawn chip to the chip stack
        self.chip_stack.append(drawn_chip)
        
        # Remove the drawn chip from the bag
        self.bag.remove(drawn_chip)
        
        return drawn_chip

    def place_colored_cube(self, regions):
        """
        Place a colored cube on either a region or the color stack.
        `regions` is a list of Region objects to choose from.
        """
        # Select a random color to place
        available_colors = [color for color, count in self.cubes.items() if count > 0]
        print(f"{self.name} has {len(available_colors)* (len(regions) + 1)} possible placement options.")    
        selected_color = random.choice(available_colors)

        # Decide randomly whether to place on stack or on a region
        place_on_stack = random.choice([True, False])

        if place_on_stack:
            # Place on the stack
            self.color_stacks[selected_color] += 1
            self.cubes[selected_color] -= 1
            #print(f"{self.name} placed a {selected_color} cube on the stack.")
        else:
            # Place on a random region
            region = random.choice(regions)
            region.add_cube(selected_color)
            self.cubes[selected_color] -= 1
            #print(f"{self.name} placed a {selected_color} cube on a random region.")


    def get_random_cubes(self, number=3):
        """Get a specified number of new random colored cubes."""
        available_colors = list(self.cubes.keys())
        for _ in range(number):
            color = random.choice(available_colors)
            self.cubes[color] += 1

In [8]:
class Region:
    def __init__(self):
        # Color stacks (each color has a list)
        # Cubes by color
        self.cubes = {
            'red': 0,
            'blue': 0,
            'green': 0,
            'pink': 0,
            'yellow': 0
        }

        # Place one random cube on one stack at init
        available_colors = list(self.cubes.keys())
        color = random.choice(available_colors)
        self.cubes[color] += 1
        
        # Castle count
        self.castles = 0

        # Owner name (None if no castle placed yet)
        self.owner = None

    def add_cube(self, color):
        """Add a cube to the specified color stack."""
        self.cubes[color] += 1
    
    def merge(self, other_region):
        """
        Merge this region with another if they have the same owner.
        Sum cubes and castles. Assumes both have the same owner.
        """
        for color in self.cubes:
            self.cubes[color] += other_region.cubes[color]
        self.castles += other_region.castles
        # Ownership remains unchanged
    


    def check_majorities(self, players, loyalty):
        """
        Determine which player has the majority influence over the region,
        and update ownership if needed.
        """
        # Map from player name to player object
        player_map = {player.name: player for player in players}
        count = {player.name: 0 for player in players}

        # Count influence based on color ownership
        for color in self.cubes:
            owner = loyalty.color_owners.get(color)
            if owner:
                count[owner] += self.cubes[color]

        #print(count)  # Optional debug output

        max_value = max(count.values())
        if max_value == 0:
            return None  # No influence

        # Determine top players by name
        top_names = [name for name, value in count.items() if value == max_value]

        if len(top_names) == 1:
            new_owner_name = top_names[0]
            new_owner = player_map[new_owner_name]
            self.update_owner_and_castles(new_owner, loyalty)
        else:
            # Tie - no ownership change
            return None


    def update_owner_and_castles(self, new_owner, loyality):
        """
        Update the region's owner and handle castle count changes.
        """
        current_owner = self.owner

        if current_owner is None:
            # No current owner: assign the new owner
            self.owner = new_owner
            self.castles += 1
            new_owner.castles -= 1
            #print(f"{new_owner.name} now owns the region and places {self.castles} castles.")
        else:
            # There is an owner: change ownership
            if current_owner != new_owner:
                # Transfer ownership
                #print(f"Ownership transfer: {current_owner.name} loses the region and {new_owner.name} becomes the new owner.")
                
                # Update castles: the old owner gets their castles added, the new owner gets 1 castle
                current_owner.castles += self.castles  # Old owner gets their castles back
                new_owner.castles -= self.castles  # New owner loses one castle
                self.owner = new_owner  # Change ownership
                #print(f"{new_owner.name} now owns the region and places {self.castles} castles.")

class Karl:
    def __init__(self):
        self.region_index = random.randint(0,15)  # Pointer to the index in the regions list

    def move_to(self, new_index):
        """Set Karl's pointer to a new region index."""
        self.region_index = new_index

class Loyalty:
    def __init__(self):
        # Dictionary to store the player names for each color (empty at first)
        self.color_owners = {
            'red': None,
            'blue': None,
            'green': None,
            'pink': None,
            'yellow': None
        }

    def calculate_ownership(self, players):
        """
        Calculate who has the most cubes for each color and assign the ownership.
        If there is a tie, ownership remains unchanged.
        """
        for color in self.color_owners:
            max_cubes = 0
            owners = []

            # Find players with the highest cube count for this color
            for player in players:
                count = player.color_stacks[color]
                if count > max_cubes:
                    max_cubes = count
                    owners = [player.name]
                elif count == max_cubes and count > 0:
                    owners.append(player.name)

            # Assign ownership only if there's a single top player
            if len(owners) == 1:
                self.color_owners[color] = owners[0]
            else:
                # Tie or no cubes: keep existing owner
                pass


def merge_regions(regions, karl_region_index):
    """
    Merge the region at `karl_region_index` with adjacent regions if they have the same owner.
    Updates the regions list in-place by removing one region and merging it into the other.
    """
    # Get the next and previous region indices
    next_index = (karl_region_index + 1) % len(regions)
    prev_index = (karl_region_index - 1) % len(regions)
    
    # Store merge candidates
    merge_candidates = []

    # Check if the regions should be merged (same owner)
    if regions[karl_region_index].owner is not None and regions[karl_region_index].owner == regions[next_index].owner:
        merge_candidates.append((karl_region_index, next_index))

    if regions[karl_region_index].owner is not None and regions[prev_index].owner == regions[karl_region_index].owner:
        merge_candidates.append((prev_index, karl_region_index))

    # Sort pairs so that the higher index is merged first (to avoid issues when deleting regions)
    for idx1, idx2 in sorted(merge_candidates, key=lambda pair: -max(pair)):
        region1 = regions[idx1]
        region2 = regions[idx2]

        # Merge the regions
        if region1.owner == region2.owner:
            # Perform the merge operation
            region1.merge(region2)

            # Remove the merged region
            del regions[max(idx1, idx2)]  # Delete the higher index first
            del regions[min(idx1, idx2)]  # Delete the lower index second

            # Add the merged region back into the list
            regions.append(region1)

            print(f"Regions at index {idx1} and {idx2} merged into one.")



### Initialization
2 Players and 15 Regions are kept in arrays

In [9]:
players = []
jonas = Player("Jonas")
christoph = Player("Christoph")
players.append(jonas)
players.append(christoph)
regions = [Region() for _ in range(15)]
loyality = Loyalty()
karl = Karl()


### Game play

As long as both players have castles left and the number of regions is above 4:
- each player draws a chip that defines their playing order
- every player places 3 cubes (and the ownerships of Loyality are recalculated)
- Karl moves a number of regions (and the owner of that region is changed, if needed)
- if needed, regions are merged
- the player redraws 3 cubes


In [10]:
while len(regions) > 4 and players[0].castles > 0 and players[1].castles > 0:
    #decide order of moves
    drawn_chip = players[0].draw_chip()  
    drawn_chip = players[1].draw_chip(drawn_chip)
    if players[0].chip_stack[-1] > players[1].chip_stack[-1]:
        players[0], players[1] = players[1], players[0]
    #move
    for player in players:
        for _ in range(3):
            player.place_colored_cube(regions) # place three cubes
        loyality.calculate_ownership(players) #recalculate loyalities
        move = random.randint(1, player.chip_stack[-1])
        karl.move_to((karl.region_index + move)%len(regions)) #move karl
        regions[karl.region_index].check_majorities(players, loyality) #check the majorities of the region karl's at
        merge_regions(regions, karl.region_index) #merge regions if possible
        player.get_random_cubes() #draw new cubes
    print(f"Regions: {len(regions)}, Castles Jonas: {jonas.castles}, Castles Christoph: {christoph.castles}")

Jonas has 80 possible placement options.
Jonas has 64 possible placement options.
Jonas has 64 possible placement options.
Christoph has 48 possible placement options.
Christoph has 48 possible placement options.
Christoph has 48 possible placement options.
Regions: 15, Castles Jonas: 10, Castles Christoph: 10
Christoph has 80 possible placement options.
Christoph has 64 possible placement options.
Christoph has 48 possible placement options.
Jonas has 48 possible placement options.
Jonas has 48 possible placement options.
Jonas has 32 possible placement options.
Regions: 15, Castles Jonas: 8, Castles Christoph: 10
Christoph has 64 possible placement options.
Christoph has 64 possible placement options.
Christoph has 48 possible placement options.
Jonas has 64 possible placement options.
Jonas has 48 possible placement options.
Jonas has 48 possible placement options.
Regions: 15, Castles Jonas: 8, Castles Christoph: 8
Christoph has 48 possible placement options.
Christoph has 48 possi